# Notebook on Model Evaluation

In [38]:
import os
import re
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
import itertools
import matplotlib.pyplot as plt
import tensorflow as tf
import torch
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score, recall_score, roc_curve
from keras.models import load_model
from PIL import Image

In [4]:
# Your DataFrame loading and regex patterns
#df = pd.read_csv('data/train.csv')
#test_cases = ['2', '6', '7', '9', '11', '15', '16', '140', '145', '146', '147', '148', '149', '154', '156']
#GET_CASE = re.compile(r"case[0-9]{1,3}")
#GET_CASE_AND_DATE = re.compile(r"case[0-9]{1,3}_day[0-9]{1,3}")
#GET_SLICE_NUM = re.compile(r"slice_[0-9]{1,4}")
#IMG_SHAPE = re.compile(r"_[0-9]{1,3}_[0-9]{1,3}_")
#
#df_out = pd.DataFrame(columns=df.columns)#
#
#def take_test_cases(row):
#    # Iterate over the rows of the DataFrame
#    case = GET_CASE.search(row["id"])[0][4:]  # Assuming the case number is in the first column
#    if case in test_cases:
#        #print(row)
#        return [row['id'], row['class'], row['segmentation']]
#    else:
#        #return [np.NaN, np.NaN, np.NaN]
#        pass
#
##df_out[['id', 'class', 'segmentation']] = df.apply(take_test_cases, axis=1, result_type='expand')
##df_out.dropna().to_csv('tmp.csv')
#df_out = df.apply(take_test_cases, axis=1, result_type='expand')
#df_out.to_csv('tmp.csv')
##df_out.dropna(axis=0)#df_out.head()
## Use boolean indexing to select rows where the first column matches any element in test_cases
##mask = df.iloc[:, 0].str.extract(GET_CASE)[0][4:].isin(test_cases)
##mask
##mask = df.iloc[:, 0].str.extract(GET_CASE)[0].str[0].isin(test_cases)
##df_out = df[mask]
# Print the resulting DataFrame
#print(df_out.head())

# Constants and Hyperparameters

In [5]:
# Define hyperparameters and constants
MEAN = 0.136
STD = 0.178
IM_SIZE = 288
BATCH_SIZE_CLF = 32
BATCH_SIZE_SGM = 2
@dataclass
class InferenceConfig:
    BATCH_SIZE:  int = 10
    NUM_BATCHES: int = 2

In [8]:
# Define input directory
ROOT_DIR = "2p5d_stride1_all_v2"
IMG_DIR = os.path.join(ROOT_DIR, "valid")
MSK_DIR = os.path.join(ROOT_DIR, "valid", "masks")
#IMG_DIR = "2p5d_stride1_all_v2/valid/images"
#MASK_DIR = "2p5d_stride1_all_v2/valid/masks"
#CASE_FOLDERS = os.listdir(IMG_DIR)
MODEL_DIR = "models"

# Evaluation Functions

In [39]:
# The predicted segmentation masks, typically output by a segmentation model.
# It's expected to have a shape of (batch_size, height, width, num_classes)
def dice_coef(predictions, ground_truths, num_classes=2, dims=(1, 2), smooth=1e-8):
    """Smooth Dice coefficient"""

    # (batch_size, num_classes, height, width).
    ground_truth_oh = F.one_hot(ground_truths, num_classes=num_classes)
    # (batch_size, height, width, num_classes)
    # permute(0, 2, 3, 1) to change the dimensions to match the one-hot
    # encoded ground truth for element-wise multiplication in the next step.
    prediction_norm = F.softmax(predictions, dim=1).permute(0, 2, 3, 1)

    # (batch_size, num_classes)
    intersection = (prediction_norm * ground_truth_oh).sum(dim=dims)
    # (batch_size, num_classes)
    summation = prediction_norm.sum(dim=dims) + ground_truth_oh.sum(dim=dims)
    # (batch_size, num_classes)
    dice = (2.0 * intersection + smooth) / (summation + smooth)
    dice_mean = dice.mean()

    return dice_mean

# Model 1: Classification + Segmentation

## Load Models

In [9]:
# Load the classification model
class_model_name = os.path.join(MODEL_DIR, "resnet50v2_nn256_lr0001_relu_batch32_epoch30_v2_2p5d.keras")
class_model = load_model(class_model_name)

## Prepare Loader

In [10]:
# Function for custom normalization
def custom_normalization(image):  
    image = image / 255.0
    image = (image - MEAN) / STD
    return image
    
# Image preprocessing
test_datagen = ImageDataGenerator(
    preprocessing_function = custom_normalization)

# Define the test set
test_generator = test_datagen.flow_from_directory(
    IMG_DIR,
    target_size=(IM_SIZE, IM_SIZE),
    batch_size=BATCH_SIZE_CLF,
    #class_mode=None,
    classes=['images'],
    shuffle=False
)

2p5d_stride1_all_v2\valid
Found 6560 images belonging to 1 classes.


## Make Predictions

In [11]:
#images = load_images(IMG_DIR, (IM_SIZE, IM_SIZE))
#y_pred = class_model.predict(images).reshape(-1)
y_pred = class_model.predict(test_generator).reshape(-1)

C:\Users\Sergio\.venv_common\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


205/205 ━━━━━━━━━━━━━━━━━━━━ 7168s 35s/step


In [14]:
THR = 0.1586085855960846
MASK = y_pred > THR

In [37]:
# store in list the two image paths correspoding to the classification output
image_list = os.listdir(os.path.join(IMG_DIR, "images"))
image_paths_clf1 = [b for a, b in zip(MASK, image_list) if a]
image_paths_clf0 = [b for a, b in zip(MASK, image_list) if not(a)]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# load the model
checkpoint_path = "static/ckpt_009.ckpt"
model = load_model(checkpoint_path)

# predict!
predictions = inference(model, image_paths_clf1, device=device)

ValueError: File format not supported: filepath=static/ckpt_009.ckpt. Keras 3 only supports V3 `.keras` files and legacy H5 format files (`.h5` extension). Note that the legacy SavedModel format is not supported by `load_model()` in Keras 3. In order to reload a TensorFlow SavedModel as an inference-only layer in Keras 3, use `keras.layers.TFSMLayer(static/ckpt_009.ckpt, call_endpoint='serving_default')` (note that your `call_endpoint` might have a different name).

In [ ]:
@torch.inference_mode()
def inference(model, image_paths, device="cpu"):
    batch_size = InferenceConfig.BATCH_SIZE
    num_images = len(image_paths)
    num_batches_to_process = (num_images + batch_size - 1) // batch_size
    results = []
    #overlay_paths = []
    for batch_idx in range(num_batches_to_process):
        start_idx = batch_idx * batch_size
        end_idx = min((batch_idx + 1) * batch_size, len(image_paths))
        batch_images = []
        for path in image_paths[start_idx:end_idx]:
            # Load image from path
            image = Image.open(path).convert("RGB")
            # Convert image to numpy array
            image_np = np.array(image)
            # Convert image to tensor -- and make sure a float and not double!!
            image_tensor = torch.tensor(image_np / 255.0, dtype=torch.float).permute(2, 0, 1).to(device)
            batch_images.append(image_tensor)
        batch_images = torch.stack(batch_images)
        model.eval()
        # Perform inference
        with torch.no_grad():
            predictions = model(batch_images)
        pred_all = predictions.logits.argmax(dim=1)

        # He we can compute the Dice_coeff and accumulate
        
        # Append batch of predicted masks to the list
        all_predictions.append(pred_all.cpu().numpy())

    # Concatenate all predicted masks into a single NumPy array
    all_predictions = np.concatenate(all_predictions)

    return all_predictions